# Imports

In [1]:
import nltk                             
from nltk.corpus import twitter_samples
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from transformers import InputExample, InputFeatures
from transformers import BertTokenizer, TFBertForSequenceClassification


In [4]:
from sklearn.model_selection import train_test_split

In [6]:
data = pd.read_feather("../data/movie_reviews_4k.feather")

In [7]:
data.shape

(4000, 2)

## Train test split

In [8]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=0)

In [9]:
data_train.reset_index(inplace=True, drop=True)
data_test.reset_index(inplace=True, drop=True)

## Load BERT model and Tokenizer

In [10]:

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Prepare inputs

In [11]:
def convert_data_to_examples(data, data_col, label_col): 
    examples= data.apply(lambda x: InputExample(
        guid=None, 
        text_a = x[data_col], 
        text_b = None,
        label = x[label_col]), axis = 1)      
    
    return examples

In [12]:
examples_train = convert_data_to_examples(data_train, "text", "label")
examples_test = convert_data_to_examples(data_test, "text", "label")

In [13]:
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )



In [14]:
len(examples_train)

3200

In [15]:
dataset_train = convert_examples_to_tf_dataset(examples_train, tokenizer, max_length=128)
dataset_test = convert_examples_to_tf_dataset(examples_test, tokenizer, max_length=128)

C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2126: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:
dataset_train = dataset_train.shuffle(100).batch(32)

In [17]:
dataset_test = dataset_test.shuffle(100).batch(32)

In [18]:
%%time

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
history = model.fit(dataset_train, epochs=5, verbose=1,  validation_data=dataset_test)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
    100/Unknown - 722s 7s/step - loss: 0.4642 - accuracy: 0.7922WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config obj

In [19]:
history.history

{'loss': [0.4641569256782532,
  0.2591298818588257,
  0.1340349316596985,
  0.09284628927707672,
  0.06756050884723663],
 'accuracy': [0.792187511920929,
  0.9012500047683716,
  0.9590625166893005,
  0.9706249833106995,
  0.9821875095367432],
 'val_loss': [0.34040001034736633,
  0.35560932755470276,
  0.41056814789772034,
  0.4552919268608093,
  0.6499009132385254],
 'val_accuracy': [0.8500000238418579,
  0.8712499737739563,
  0.8650000095367432,
  0.856249988079071,
  0.8475000262260437]}

In [26]:
data.text[0]

"I wanted to vote zero or lower. I loved the commentary. It IS the worst movie ever made and 'unendurable' is the perfect word for it, unless there is something worse that Roget never thought of. I am also at a loss to think of anything negative enough to accurately describe Bo Derek. The best that could be said of her is, she's consistent."

In [27]:
pred_sentences  =[ "I absolutely hate this movie, total dissaster", 
                  "Most beaytiful movie ever, I watched it 10 times, very good"]

In [28]:
def classify_sentences(sentences):
    sententes_tokenized = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
    model_outputs = model(sententes_tokenized)
    sentiment_proba =  tf.nn.softmax(model_outputs[0], axis=-1)
    sentiment_class = tf.argmax(sentiment_proba, axis=1)
    sentiment_labels = [ 'Negative', 'Positive']
    sentiments = [sentiment_labels[i] for i in sentiment_class ]
    
    return sentiment_proba, sentiments

In [31]:
classify_sentences(pred_sentences )

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[0.9948179 , 0.00518212],
        [0.00184642, 0.9981535 ]], dtype=float32)>,
 ['Negative', 'Positive'])

In [32]:
data_train.label.mean()

0.48625